In [86]:
# 1. Import dependencies
import requests
import pymongo

import pandas as pd
import numpy
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint

</br>
<center><font size=8, color="orange", face="Georgia"> Scrapped State Abbreviations</font><center>

![Abbreviations-Your Dictionary](./images/abbreviations.png)

# First Source -- STATES --
### Scrape list containing name and abbreviation for each state

In [3]:
# 1. get url
url = "https://abbreviations.yourdictionary.com/articles/state-abbrev.html"

In [4]:
# 2. connect to url
response = requests.get(url)
response

<Response [200]>

In [5]:
# 3. parse html page using Beautiful Soup
soup = bs(response.text, 'lxml')

# 50 USA States

In [6]:
# 4. locate table containing all 50 state names and abbr. for 'div id'
states = soup.select('div[id^=article_body]')
for s in states:
    slt = soup.find('ul')
    #scrape names
    for a in slt:
        state_abbrev = slt.find_all('li')
        
# create list to store items
state_list = []
#loop thorugh iterable list, abstract text value and appended each to new list
for one in state_abbrev:
    state_list.append(one.text) 

# Additional Territories

In [7]:
# 5. locate list containing state and abbr.
abbrev = soup.find('div', class_='article_double')
#scrapped list
for a in abbrev:
    list_other = abbrev.find('ul')
    for a in list_other:
        others = list_other.find_all('li')
        
#loop thorugh iterable list, abstract text value and appended each to new list
for o in others:
    state_list.append(o.text) 

# Clean Data & Display DF

In [18]:
# 6. save list as data frame
states = pd.DataFrame({'state':state_list})

# split columns to separate Abbrev and Name (will use it a primary key to join tables)
state = states['state'].str.split('-', 1, expand=True)
state
#rename col to 'State', set state as new index
state_df = state.rename(columns={state.columns[0]: "State"}).set_index("State")
state_df.head()

,1
State,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


<center><font size=8, color="green", face="Georgia"> Minimum Wage for all USA States from <br /> 1968 - 2018</font><center>

![Abbreviations-Your Dictionary](./images/deptlabor.png)

# Second Source -- WAGES --
### Locate minimum wage for all states & territories for the past 50 years

In [112]:
# 1. obtain and merge wage tables for the last 10 years
#2007-2013
data4 = pd.read_html('https://www.dol.gov/whd/state/stateMinWageHis.htm',attrs={'summary': 'This is a table of minimum changes by state for 2007 to 2013.'})[0]
df4 = data4.reindex(data4.index.drop(0))
#2014-2018
data5 = pd.read_html('https://www.dol.gov/whd/state/stateMinWageHis.htm',attrs={'summary': 'This is a table of minimum changes by state for 2014 to 2017.'})[0]
df5 = data5.reindex(data5.index.drop(0))

In [121]:
# 2. merge all dfs on "State or other  jurisdiction". Note: Be mindful of spacing
# rename col for join, drop empty '...' values, TOTAL STATES :49, set index, sort descendig value
wagesdf = df4.merge(df5,on='State or other  jurisdiction').rename(index=str, columns={"State or other  jurisdiction": "State Name"}).drop(['0', '17', '23', '39', '41']).sort_values(by='State Name', ascending=True)
wagesdf.head()

,State Name,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,Alaska,7.15,7.15,7.15,7.75,7.75,7.75,7.75,7.75,8.75,9.75,9.80,9.84
2,Arizona,6.75,6.90,7.25,7.25,7.35,7.65,7.80,7.90,8.05,8.05,10.00,10.50
3,Arkansas,6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],7.50[c],8.00[c],8.50[c],8.50[c]
4,California,7.50,8.00,8.00,8.00,8.00,8.00,8.00,9.00,9.00,10.00,10.00,11.00
5,Colorado,6.85,7.02,7.28,7.24,7.36,7.64,7.78,8.00,8.23,8.31,9.30,10.20


# Merge: States & Wages

In [122]:
# List Source (Github Repo) // Manually cleanse list. 
# limitation: when merging original scrapped list 'state_df', no results displayed. Tried different ways of joining, merging, & concatinating.  
states = ["AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", 
          "GU", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "PR", "RI", 
          "SD", "TX", "VI", "UT", "VT" ,"VA", "WA", "WV", "WI", "WY"]

# 3. insert a new column called "Abbrev" with index col[0], inside wages_df using list above "states"
wagesdf.insert(0, 'Abrv', states)

In [123]:
wagesdf.head()

,Abrv,State Name,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,AK,Alaska,7.15,7.15,7.15,7.75,7.75,7.75,7.75,7.75,8.75,9.75,9.80,9.84
2,AZ,Arizona,6.75,6.90,7.25,7.25,7.35,7.65,7.80,7.90,8.05,8.05,10.00,10.50
3,AR,Arkansas,6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],7.50[c],8.00[c],8.50[c],8.50[c]
4,CA,California,7.50,8.00,8.00,8.00,8.00,8.00,8.00,9.00,9.00,10.00,10.00,11.00
5,CO,Colorado,6.85,7.02,7.28,7.24,7.36,7.64,7.78,8.00,8.23,8.31,9.30,10.20


<center><font size=8, color="green", face="Georgia"> USA CITIES</font><center>

![Kaggle](./images/kaggle.png)

# Third Source - Final merge

* We have now appended "Abbrev" names into dataframe 'wage_df' and kept "State" as the index.
* Finally, append 'wage_df' to 'USA_cities.csv' to complete merge and display final results

In [124]:
# (Source used:--KAGGLE: https://www.kaggle.com/shivamb/cities#zip_codes_states.csv)
# 1. read csv file saved as USA_cities, and rename column (key)
df= pd.read_csv("./USA_cities.csv").rename(columns={'state': 'Abrv'})
df.head()

,zip_code,latitude,longitude,city,Abrv,county
0,501,40.922326,-72.637078,Holtsville,NY,Suffolk
1,544,40.922326,-72.637078,Holtsville,NY,Suffolk
2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas
3,602,18.393103,-67.180953,Aguada,PR,Aguada
4,603,18.455913,-67.145780,Aguadilla,PR,Aguadilla


# Initialize, Connect, Define & Push to MongoDB

In [146]:
# 1. initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [147]:
# 2. define database and collection
db = client.metricsusa_db
collection = db.usa_citiwages

In [148]:
# 3. cleanse final data: merge dfs, set 'city' as index, drop duplicates,  
data_clean = df.merge(wagesdf, on="Abrv").set_index('city')
asd = data_clean.groupby('State Name').count()
#final count #48 STATES
asd.count()

zip_code     48
latitude     48
longitude    48
Abrv         48
county       48
2007         48
2008         48
2009         48
2010         48
2011         48
2012         48
2013         48
2014         48
2015         48
2016         48
2017         48
2018         48
dtype: int64

In [145]:
# 4. save as json dict in decending order by 'city'
citiwage_jfile = data_clean.to_dict('index')
#pprint(citiwage_jfile)

In [150]:
# 5. push josn file to collection 'usa_citiwages' in Mongo DB 'metricsusa_db' 
collection.insert_one(citiwage_jfile)

# Output file (CVS, comma delimited // (Excel)) 

In [151]:
# 1. output results in csv file (for Vika's reference)
data_clean.to_csv("US_Citiwages.csv", index=False)